# Create country profiles

In [32]:
import csv
import json
import urllib3  # allows to access a URL with python
import urllib
import re
import statistics
import math
import os

### User parameters

In [33]:
release = '2019.Q1.G.03' # Make sure to have the correct release here

dir_path = os.path.dirname(os.path.realpath('__file__'))
print(dir_path)

wd_dir = r'../../data/unsd/countryProfiles'
print('data inputs dir: ' + wd_dir)

C:\Users\L.GonzalezMorales\Documents\GitHub\FIS4SDGs\notebooks\unsdCountryProfiles
data inputs dir: ../../data/unsd/countryProfiles


## Utilities

#### Commercial rounding

In [34]:
def round_KFM(x, n):
    posneg = math.copysign(1, x)
    z = abs(x)*10**n
    z = z + 0.5
    z = math.trunc(z)
    z = z/10**n
    result = z * posneg
    return result
    
print(round_KFM(3.5123,2))
print(round_KFM(4.0,1))

3.51
4.0


#### Regular expression to capture numeric values (including those in scientific notation)
The regex is

```
-?      # an optional -
\d+     # a series of digits
(?:     # start non capturing group
  \.    # a dot
  \d+   # a series of digits
)?      
(?:     # start non capturing group
  e     # "e"
  -?    # an optional -
  \d+   # digits
)?
```


In [35]:
def numeric_part(v):
    numeric_part_f = re.compile(r'-?\d+(?:\.\d+)?(?:e-?\d+)?')
    x = numeric_part_f.findall(v)
    if len(x) > 0:
        return float(x[0])
    else:
        return None


In [36]:
print(numeric_part('<4'))
print(numeric_part('abs'))
print(numeric_part('-23.2e2'))
print(numeric_part('-23.2e2 or more'))

print(numeric_part('0.01'))

4.0
None
-2320.0
-2320.0
0.01


#### Compute a hash of a dictionary

In [37]:
def dict_hash(d):
    out = hashlib.md5()
    for key, value in d.items():
        out.update(key.encode('utf-8'))
        out.update(value.encode('utf-8'))
    return out.hexdigest()


#### Get unique dictionaries in a list

In [38]:
def unique_dicts(dictionary_list):

    uniques_map = {}

    for d in dictionary_list:
        uniques_map[dict_hash(d)] = d

    return list(uniques_map.values())


#### Extract subset of key-value pairs from Python dictionary object

In [39]:
def subdict_list(dict_list, keys_list, exclude = False):
    sub_d_list = []
    if exclude:
        for d in dict_list:
            sub_d= {k: d[k] for k in d.keys() if k not in keys_list}
            sub_d_list.append(sub_d)
    else:
        for d in dict_list:
            sub_d= {k: d[k] for k in keys_list}
            sub_d_list.append(sub_d)
    
    return sub_d_list




#### Get a dict from a list based on something inside the dict

In [40]:
def select_dict(dict_list, k, v):
    selected = []
    for d in dict_list:
        if d[k] == v:
            selected.append(d)
    return selected

### Read fact-builder conditions

In [42]:
fact_builder = []
with open(wd_dir + r'/input/CountryProfileBuilder_new.csv', newline = '') as countryProfileBuilder:                                                                                          
    country_profile_builder = csv.DictReader(countryProfileBuilder, delimiter=',')
    for row in country_profile_builder:
        fact_builder.append(dict(row))
        
print(fact_builder[1])


{'countryProfile': '1', 'goal': '1', 'target': '1.2', 'indicator': '1.2.1', 'seriesCode': 'SI_POV_NAHC', 'dashboardId': '62f44e7da0864b1eb6f9d4bb5f9d2f54', 'seriesDesc': 'Proportion of population living below the national poverty line (%)', 'sliceDesc': 'Proportion of population living below the national poverty line (percent)', 'ageCode': '', 'locationCode': '_T', 'sexCode': '', 'boundsCode': '', 'educationLevelCode': '', 'typeOfProductCode': '', 'typeOfMobileTechnologyCode': '', 'typeOfSpeedCode': '', 'zero.removal': '1', 'Text.type': '1', 'unit1': '%', 'unit2': '', 'DA2.1': 'The proportion of population living below the national poverty line was approximately', 'DA2.2': '', 'DA3.1': 'The proportion of population living below the national poverty line', 'DA3.2': '', 'Down': ' declined from ', 'Up': ' increased from ', 'Threshold': ''}


### Create list of countries

In [43]:
countryArray = []
with open(r'..\..\globalResources\refAreas.txt', newline = '') as countryListXY:                                                                                          
    countryListXY = csv.DictReader(countryListXY, delimiter='\t')
    for row in countryListXY:
        countryArray.append(dict(row))
        
print(countryArray[1])

    
print("Total number or countries: ", len(countryArray))

{'X': '20.06660928', 'Y': '41.13897007', 'ISO3': 'ALB', 'M49': '8', 'areaName': 'Albania', 'UN_Member': '1', 'Country_Profile': '1'}
Total number or countries:  248


### Get the list of goals, targets, indicators and series

In [44]:
with open(r'..\..\globalResources\metadata.json') as json_file:  
    metadata = json.load(json_file)

### Get catalogue of UNSD items on SDG open data groups

In [45]:
with open(r'..\..\data\unsd\unsd_catalogue.json') as json_file:  
    hub_catalogue = json.load(json_file)
    
hub_catalogue[0]['series'][0]

{'seriesCode': 'SI_POV_EMP1',
 'seriesName': 'Employed population below international poverty line  by sex and age (%)',
 'release': '2019.Q1.G.03',
 'id': 'e8e11539dd9a4293b502b076d041133d',
 'owner': 'unstats_admin',
 'created': 1560397341000,
 'modified': 1560542265180,
 'title': 'Indicator 1.1.1: Employed population below international poverty line, by sex and age (percent)',
 'type': 'Feature Service',
 'tags': ['poverty line',
  'poverty',
  'standard of living',
  'basic needs',
  'decent work',
  '2019.Q1.G.03'],
 'url': 'https://services7.arcgis.com/gp50Ao2knMlOM89z/arcgis/rest/services/SI_POV_EMP1_1_1_1_2019Q1G03/FeatureServer'}

## Create country profiles

In [46]:
countryArray[0].keys()
countryArray[0]

{'X': '66.02688198',
 'Y': '33.83160199',
 'ISO3': 'AFG',
 'M49': '4',
 'areaName': 'Afghanistan',
 'UN_Member': '1',
 'Country_Profile': '1'}

## Build fact text

In [47]:
def build_fact(text_type, conditions, da2_1, da2_2, da3_1, unit_1, unit_2, value_y_min, y_min, value_y_max, y_max, prog, prog_10, prog_12, prog_15, prog_mmr_max):
    
    fact_text = ''
    fact_values = []
    fact_units = []
    fact_years = []
    
    if text_type =='1':
        if conditions:
            fact_text = da3_1 + prog + "<span class='fact-value'>" + str(value_y_min) + "</span>  <span class='fact-unit'>" + unit_1  + "</span>  in  <span class='fact-year'>" + str(y_min) + "</span> to <span class='fact-value'>" + str(value_y_max) + unit_1 + "</span> in <span class='fact-year'>" + str(y_max)  + "</span>."
            fact_values = [ str(value_y_min), str(value_y_max)]
            fact_units =  [ unit_1, unit_1]
            fact_years =  [ str(y_min), str(y_max)]
        else:
            fact_text = da2_1 +  "<span class='fact-value'>" + str(value_y_max) + "</span> <span class='fact-unit'>" + unit_1 + "</span> in <span class='fact-year'>" + str(y_max)  + "</span>."
            fact_values = [ str(value_y_max)]
            fact_units = [ unit_1]
            fact_years =  [ str(y_max)]

    elif text_type == '2':
        fact_text = "In <span class='fact-year'>" + str(y_max)  + "</span>, <span class='fact-value'>" + str(value_y_max) + "</span> <span class='fact-unit'>" + unit_1 + "</span> " + da2_1
        fact_values = [ str(value_y_max)]
        fact_units = [ unit_1]
        fact_years =  [ str(y_max)]

    elif text_type == '3':
        fact_text = "In <span class='fact-year'> " + str(y_max) +  "</span>, " + da2_1 + "<span class='fact-value'> " + str(value_y_max) + "</span> <span class='fact-unit'>" + unit_1 + "</span> " + da2_2
        fact_values = [ str(value_y_max)]
        fact_units = [ unit_1]
        fact_years =  [ str(y_max)]

    elif text_type == '4':
        fact_text = da2_1+  "<span class='fact-value'>"  + str(value_y_max) +  "</span> in <span class='fact-year'> " + str(y_max) +  "</span> " + ", meaning "  + str(float(value_y_max) * 100) +  da2_2 + "." 
        fact_values = [ str(value_y_max)]
        fact_units = [ unit_1]
        fact_years =  [ str(y_max)]

    elif text_type == '7':
        fact_text = da2_1 +  "<span class='fact-value'> " + str(value_y_max) +  "</span> <span class='fact-unit'>" + unit_1 +  "</span> "  + " in "+  "<span class='fact-year'> "  + str(y_max)+  "</span> "  + "."
        fact_values = [ str(value_y_max)]
        fact_units = [ unit_1]
        fact_years =  [ str(y_max)]

    elif text_type == '8':
        fact_text = "In "+  "<span class='fact-year'> "  + str(y_max)+  "</span>, <span class='fact-value'> " + str(value_y_max)+  "</span> "  + " <span class='fact-unit'>" + unit_1 +  "</span> "+ da2_1 + "."
        fact_values = [ str(value_y_max)]
        fact_units = [ unit_1]
        fact_years =  [ str(y_max)]

    elif text_type == '9':
        if conditions:
            fact_text = "In "+  "<span class='fact-year'> " + str(y_max) +  "</span>, " + da2_1 +  "<span class='fact-value'> "+ str(value_y_max) +  "</span> " + " <span class='fact-unit'>" + unit_1+  "</span>, " + prog +  "<span class='fact-value'> "+ str(value_y_min) +  "</span> <span class='fact-unit'>" + unit_2 + "</span> in <span class='fact-year'> " + str(y_min) +  "</span> "
            fact_values = [ str(value_y_min), str(value_y_max)]
            fact_units = [unit_1, unit_1]
            fact_years =  [ str(y_min), str(y_max)]
        else:
            fact_text = "In "+  "<span class='fact-year'> " + str(y_max)+  "</span>, " + da2_1 +  "<span class='fact-value'> "+ str(value_y_max) +  "</span> "  + " <span class='fact-unit'>" + unit_1+  "</span>."
            fact_values = [ str(value_y_max)]
            fact_units = [ unit_1]
            fact_years =  [ str(y_max)]

    elif text_type == '10':
        fact_text =  "In " +  "<span class='fact-year'> "+ str(y_max) +  "</span>, " +  da2_1 +  "<span class='fact-value'> "+ str(value_y_max) +  "</span> "  + " <span class='fact-unit'>" + unit_1 +  "</span>. " + prog_10 
        fact_values = [ str(value_y_max)]
        fact_units = [ unit_1]
        fact_years =  [ str(y_max)]

    elif text_type == '11':
        fact_text = "In "+  "<span class='fact-year'> " + str(y_max)+  "</span>, " +  da2_1 +  "<span class='fact-value'> " + str(value_y_max) +  "</span> "  + " <span class='fact-unit'>" + unit_1 +  "</span>. "
        fact_values = [ str(value_y_max)]
        fact_units = [ unit_1]
        fact_years =  [ str(y_max)]

    elif text_type == '12':
        fact_text = "As of " +  "<span class='fact-year'> "+ str(y_max) +  "</span>, " + country_name + prog_12 + "."
        fact_values = [ str(value_y_max)]
        fact_units = [ unit_1]
        fact_years =  [ str(y_max)]

    elif text_type == '13':
        fact_text =  "In " +  "<span class='fact-year'> "+ str(y_max)+  "</span>, " +  da2_1 +  "<span class='fact-value'> " + str(value_y_max) +  "</span> " + " <span class='fact-unit'>" + unit_1 +  "</span> "+ da2_2 + "."
        fact_values = [ str(value_y_max)]
        fact_units = [ unit_1]
        fact_years =  [ str(y_max)]

    elif text_type == '14':
        if conditions:
            fact_text = da3_1 + prog + "<span class='fact-value'>" + str(value_y_min) +  "</span> <span class='fact-unit'>" + unit_1 +  "</span> in <span class='fact-year'> "+ str(y_min)+  "</span> to <span class='fact-value'> " + str(value_y_max) +  "</span> <span class='fact-unit'>"  + unit_1 +  "</span> in <span class='fact-year'> " + str(y_max)+  "</span>."
            fact_values = [ str(value_y_min), str(value_y_max)]
            fact_units = [ unit_1, unit_1]
            fact_years =  [ str(y_min), str(y_max)]
        else:
            fact_text = da2_1 +  "<span class='fact-value'> "+ str(value_y_max)+  "</span> <span class='fact-unit'>" + prog_mmr_max + unit_1  +  "</span> in <span class='fact-year'> " + str(y_max) +  "</span>."
            fact_values = [ str(value_y_max) + prog_mmr_max]
            fact_units = [ unit_1]
            fact_years =  [ str(y_max)]

    elif text_type == '15':
        if conditions:
            fact_text =  "In "+  "<span class='fact-year'> " + str(y_max) +  "</span> , " +  da2_1 +  "<span class='fact-value'> "+ str(value_y_max) +  "</span> <span class='fact-unit'>" + unit_1 + "</span>," + prog + prog_15 + " in <span class='fact-year'> " + str(y_min) +  "</span>."
            fact_values = [ str(value_y_min), str(value_y_max)]
            fact_units = [ unit_1, unit_1]
            fact_years =  [ str(y_min), str(y_max)]
        else:
            fact_text =  "In "+  "<span class='fact-year'> " + str(y_max)+  "</span> , " +  da2_1 +  "<span class='fact-value'> "+ str(value_y_max)+  "</span> <span class='fact-unit'>" + unit_1 + "</span>."
            fact_values = [ str(value_y_max)]
            fact_units = [ unit_1]
            fact_years =  [ str(y_max)]
    
    return {'fact_text': fact_text,
            'fact_values': fact_values,
            'fact_units': fact_units,
            'fact_years': fact_years
           }


## Information about status of progress: decline/increase?

In [48]:
def prog_info (value_y_min_num, value_y_max_num, down, up, unit_1):
    
    prog = None
    prog_10 = None
    prog_12 = None
    prog_15 = None
    prog_mmr_min = None
    prog_mmr_max = None
    
    #-----------------------------------------
    if value_y_min_num and value_y_max_num:
        if (value_y_min_num > value_y_max_num):
            prog = down
            # Example: "declined from", "down from"
        elif (value_y_min_num < value_y_max_num):
            prog = up
            # Example: "increased from", "up from"
        else: prog = ""
    else:
        prog = ""

    #print(prog)

    #--------
    # prog.15
    #--------
    if value_y_min_num < 0.01:
         prog_15 = "nearly no coverage"
    else:
        prog_15 = str(value_y_min_num) + unit_1

    #--------
    # prog.10
    #--------
    if value_y_max_num:
        if value_y_max_num < 10:
            prog_10 = down
        else:
            prog_10 = up
    else:
        prog_10 = ''

    #--------
    # prog.12
    #--------
    
    if value_y_max_num is not None:
        if value_y_max_num > 0:
            prog_12 = up 
        else:
            prog_12 = down
    else:
        prog_12 = ''


    #-------------
    # prog_mmr_min
    #-------------
    if value_y_min_num:
        if value_y_min_num > 1: 
            prog_mmr_min = "deaths"
        else:
            prog_mmr_min = "death"
    else:
        prog_mmr_min = ""
    
    #-------------
    # prog_mmr_max
    #-------------
    
    if value_y_max_num:
        if value_y_max_num > 1: 
            prog_mmr_max = "deaths" 
        else:
            prog_mmr_max = "death"
    else:
        prog_mmr_max = ""

    #print("min = ",value_y_min_num)
    #print("max = ",value_y_max_num)
    #print("prog = ",prog)
    #print(prog_15)
    #print(prog_10)
    #display(print(prog_12))
    #print(prog_mmr_min)
    #print(prog_mmr_max)
    
    return {'prog' : prog,
            'prog_10' : prog_10,
            'prog_12' : prog_12,
            'prog_15' : prog_15,
            'prog_mmr_min' : prog_mmr_min,
            'prog_mmr_max' : prog_mmr_max
           }


In [50]:
count_country = 0

for this_country in countryArray:
    #if this_country['M49'] != '8':
    #    continue
    
    if this_country['Country_Profile'] != '1':
        continue
   
    count_country += 1
    
    country_profile = {}
    country_profile['release'] = release 

    country_code = str(this_country['M49'])
    country_name = this_country['areaName']
    
    country_profile['country_code'] = country_code
    country_profile['country_name'] = country_name
    country_profile['X'] = this_country['X']
    country_profile['Y'] = this_country['Y']
    country_profile['ISO3'] = this_country['ISO3']
    country_profile['UNMember'] = this_country['UN_Member']
    country_profile['CountryProfile'] = this_country['Country_Profile']
    
    print("Building country profile for ", country_name, " - (", count_country, ")")
    
    count_fact = 0
    
    goals = []
    
    for g in metadata:
        #print("- in goal " + g['code'])
        
        goal = {}
        
        goal['goalCode'] = g['code']
            
        targets = []
    
        for t in g['targets']:
            #if t['code'] 1= '2.2':
            #    continue
            #print("- - in target " + t['code'])
            
            target = {}
                
            target['targetCode'] = t['code']

            indicators = []
                
            for i in t['indicators']:
                #print("- - - in indicator " + i['reference'])
                
                 
                #if i['reference'] != '2.2.2':
                #    continue

                
                indicator = {}
                indicator['indicatorCode'] = i['reference']
                indicator['facts'] = []
                
                facts = []
                
                if 'series' in i.keys():
                    for s in i['series']:
                        
                        #print("- - - - in series " + s['code'])
                        
                        if s['release'] == release:
                            
                            #if s['code'] != 'SG_STT_NSDSFND':
                            #    continue
                            
                            
                            for this_fact in fact_builder:
                                
                                
                                
                                if this_fact['countryProfile'] != '1' or this_fact['seriesCode'] != s['code']:
                                    continue
                                #print("- - - - - in fact " + this_fact['seriesCode'])
                                
                                count_fact += 1
                                
                                seriesCode = s['code']
                                seriesTitle = this_fact['sliceDesc']
                                
                                for h in hub_catalogue:
                                    if h['goal'] != int(this_fact['goal']):
                                        continue

                                    for hs in h['series']:

                                        if hs['seriesCode'] != this_fact['seriesCode']:
                                            continue

                                        hub = hs['id']
                                        
                                #-----------------------------------------------------------
                                # Select dimensions values that are applicable for this fact
                                #-----------------------------------------------------------

                                slice_dimensions = {d: this_fact[d] for d in this_fact.keys() &
                                                    {'ageCode', 
                                                     'locationCode', 
                                                     'sexCode', 
                                                     'boundsCode', 
                                                     'educationLevelCode', 
                                                     'typeOfProductCode', 
                                                     'typeOfMobileTechnologyCode', 
                                                     'typeOfSpeedCode'}}

                                slice_dimensions = dict((k, v) for k, v in slice_dimensions.items() if v != '')

                                
                                #-----------------------------------------------------------
                                # Colect data for this fact
                                #-----------------------------------------------------------

                                ##--- Read data for this series---#

                                filename = 'Indicator_' + this_fact['indicator'] + '_Series_' + this_fact['seriesCode'] + '.json'
                                with open(r'../../data/unsd/' + release + '/' + filename) as json_file:  
                                    data = json.load(json_file)


                                ##--- Filter data for this_country['M49'] ---#

                                data = select_dict(data['refAreas'], 'geoAreaCode', str(this_country['M49']).zfill(3))

                                data = data[0]['data']
                                
                                if len(slice_dimensions)>0:
                                    for k,v in slice_dimensions.items():
                                        data = select_dict(data, k, v)

                                #----------------------------------------------------------------
                                # Main fact calculation 
                                #----------------------------------------------------------------

                                values = []
                                years = []
                                values_is_censored = []
                                values_numeric_part = []

                                n = None      # number of observations available
                                y_min = None    # first year available
                                y_max = None    # most recent year available

                                value_y_min = None   # data value in the first year available
                                value_y_max = None   # data value in the most recent year available

                                value_y_min_num = None   # data value in the first year available
                                value_y_max_num = None   # data value in the most recent year available


                                value_median = None
                                dif_first_last = None
                                value_y_max_is_censored = None

                                prog = ""
                                prog_10 = ""
                                prog_12 = ""
                                prog_15 = ""
                                prog_mmr_min = ""
                                prog_mmr_max = ""

                                fact_text = ""

                                if len(data) > 0:

                                    for d in data:
                                        if d['value_detail'] not in ['NA', 'N', 'NULL']:
                                            values.append(d['value_detail'])
                                            years.append(d['timePeriod'])
                                            values_is_censored.append(d['value_is_censored'])
                                            values_numeric_part.append(numeric_part(d['value_detail']))
                                        
                                 
                                    for i in range(len(values)):
                                        values[i] = values[i].replace("<", "less than ")
                                        values[i] = values[i].replace(">", "more than ")
                                    
                                    for i in range(len(values)):
                                        
                                        decimal_pos = values[i].find('.')
                                        
                                        if decimal_pos == -1 and values_numeric_part[i] is not None:
                                            values_numeric_part[i] = int(values_numeric_part[i])
                                            
                                        if decimal_pos > 0:
                                            if len(values[i]) - decimal_pos > 2:
                                                values_numeric_part[i] = round_KFM(values_numeric_part[i],2)
                                                
                                    
                                    if all(v is None for v in values_numeric_part):
                                        continue


                                    #----------------------------------------------------------------
                                    # Initial auxiliary variables
                                    #----------------------------------------------------------------


                                    n = len(data)      # number of observations available
                                    y_min = min(years)    # first year available
                                    y_max = max(years)    # most recent year available

                                    value_y_min = values[years.index(min(years))]   # data value in the first year available
                                    value_y_max = values[years.index(max(years))]   # data value in the most recent year available
                                    
                                    
                                    value_y_min_num = values_numeric_part[years.index(min(years))]   # data value in the first year available
                                    value_y_max_num = values_numeric_part[years.index(max(years))]   # data value in the most recent year available
                                   
                                    x =  [i for i in values_numeric_part if i is not None]
                                    
                                    if len(x)>0:
                                        value_median = statistics.median(x)
                                    else:
                                        value_median = None
                                    #print(seriesCode)
                                    #display(values_numeric_part)
                                    #display()
                                    #display(x)
                                    #print("----------")

                                    
                                    if(values_numeric_part[years.index(min(years))] is not None and values_numeric_part[years.index(max(years))] is not None):
                                        dif_first_last = abs(values_numeric_part[years.index(min(years))] \
                                                             - values_numeric_part[years.index(max(years))])


                                    value_y_min_is_censored = values_is_censored[years.index(min(years))]
                                    value_y_max_is_censored = values_is_censored[years.index(max(years))]
                                    
                                    fact_prog = prog_info (value_y_min_num, value_y_max_num, 
                                                           down = this_fact['Down'], 
                                                           up = this_fact['Up'], 
                                                           unit_1 = this_fact['unit1'])

                                    #--------------------------------------------------------------------

                                    fact_text = ""
                                    
                                    if value_y_max_num and value_median:
                                        condition1 = dif_first_last >= 0.05 * abs(value_y_max_num)
                                        condition2 = not value_y_max_is_censored
                                        condition3 = value_y_max_num >= .25*value_median
                                        condition4 = int(y_min) < 2010
                                        condition5 = n > 1

                                        conditions = condition1 and condition2 and condition3 and condition4 and condition5

                                    else:
                                        
                                        conditions = False
                                    
                                    if value_y_min_is_censored:
                                        fact_value_y_min = value_y_min
                                    else:
                                        fact_value_y_min = str(value_y_min_num)
                                    
                                    if value_y_max_is_censored:
                                        fact_value_y_max = value_y_max
                                    else:
                                        fact_value_y_max = str(value_y_max_num)
                                    
                                    fact_elements = build_fact(text_type = this_fact['Text.type'], 
                                                               conditions = conditions, 
                                                               da2_1 = this_fact['DA2.1'], 
                                                               da2_2 = this_fact['DA2.2'], 
                                                               da3_1 = this_fact['DA3.1'], 
                                                               unit_1 = this_fact['unit1'], 
                                                               unit_2 = this_fact['unit2'], 
                                                               value_y_min = fact_value_y_min, 
                                                               y_min = y_min, 
                                                               value_y_max = fact_value_y_max, 
                                                               y_max = y_max, 
                                                               prog = fact_prog['prog'], 
                                                               prog_10 = fact_prog['prog_10'], 
                                                               prog_12 = fact_prog['prog_12'], 
                                                               prog_15 = fact_prog['prog_15'], 
                                                               prog_mmr_max = fact_prog['prog_mmr_max'])

                                    fact = {}
                                    fact['seriesCode'] = seriesCode
                                    fact['seriesTitle'] = seriesTitle
                                    fact['hub'] = hub 
                                    fact['dashboard'] = this_fact['dashboardId']
                                    fact['slice_dimensions'] = slice_dimensions
                                    fact['text_type'] = this_fact['Text.type']
                                    fact['fact_text'] = fact_elements['fact_text']
                                    fact['fact_values'] = fact_elements['fact_values']
                                    fact['fact_units'] = fact_elements['fact_units']
                                    fact['fact_years'] = fact_elements['fact_years']
                                    fact['data_years'] = years
                                    fact['data_values'] = values
                                    fact['data_is_censored'] = values_is_censored
                                    fact['data_numeric_part'] = values_numeric_part

                                    if this_fact['Text.type'] == '10':
                                        fact['preferred_visualization'] = 'threshold'
                                        fact['threshold_value'] = float(this_fact['Threshold'])
                                        fact['threshold_units'] = this_fact['unit1']
                                    elif this_fact['Text.type'] == '12':
                                        fact['preferred_visualization'] = 'boolean'
                                    elif(len(values)>3):
                                        fact['preferred_visualization'] = 'time_series'
                                    else:
                                        fact['preferred_visualization'] = 'singleton'


                                    #=======================  
                                    #print('***** apending fact ******')
                                    #display(fact)
                                    facts.append(fact)

                    if len(facts)>0:
                        #print('**** apending facts ******')
                        #display(facts)
                        indicator['facts'] = facts
                        indicators.append(indicator)
                    #display(indicators)
                
            target['indicators'] = indicators
            #display(target)
                
            if len(target['indicators'])>0:
                #print('*** apending indicators **')
                #display(indicators)
                targets.append(target)
    
            #display(targets)
    
        goal['targets'] = targets
        #display(goal)
                    
        if len(targets)>0:
            #print('** apending targets ******')
            #display(targets)
            goals.append(goal)
            #display(goals)
    
    country_profile['goals'] = goals
    #display(goals)
               
    #=======================

    with open(wd_dir + r'/output/country_profile' + str(country_code).zfill(3) + ".json", 'w') as outfile:
        json.dump(country_profile, outfile, indent=4 )    



                

Building country profile for  Afghanistan  - ( 1 )
Building country profile for  Albania  - ( 2 )
Building country profile for  Antarctica  - ( 3 )
Building country profile for  Algeria  - ( 4 )
Building country profile for  American Samoa  - ( 5 )
Building country profile for  Andorra  - ( 6 )
Building country profile for  Angola  - ( 7 )
Building country profile for  Antigua and Barbuda  - ( 8 )
Building country profile for  Azerbaijan  - ( 9 )
Building country profile for  Argentina  - ( 10 )
Building country profile for  Australia  - ( 11 )
Building country profile for  Austria  - ( 12 )
Building country profile for  Bahamas  - ( 13 )
Building country profile for  Bahrain  - ( 14 )
Building country profile for  Bangladesh  - ( 15 )
Building country profile for  Armenia  - ( 16 )
Building country profile for  Barbados  - ( 17 )
Building country profile for  Belgium  - ( 18 )
Building country profile for  Bermuda  - ( 19 )
Building country profile for  Bhutan  - ( 20 )
Building count